In [6]:

'''This script goes along the blog post
"Building powerful image classification models using very little data"
from blog.keras.io.
It uses data that can be downloaded at:
https://www.kaggle.com/c/dogs-vs-cats/data
In our setup, we:
- created a data/ folder
- created train/ and validation/ subfolders inside data/
- created cats/ and dogs/ subfolders inside train/ and validation/
- put the cat pictures index 0-999 in data/train/cats
- put the cat pictures index 1000-1400 in data/validation/cats
- put the dogs pictures index 12500-13499 in data/train/dogs
- put the dog pictures index 13500-13900 in data/validation/dogs
So that we have 1000 training examples for each class, and 400 validation examples for each class.
In summary, this is our directory structure:
```
data/
    train/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
This code is designed to run with dlws/tutorial-tensorflow:1.5

```
'''


from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import *
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.callbacks import *
import os.path

from keras.applications.densenet import DenseNet201
from keras.applications.resnet50 import ResNet50

import tensorflow as tf
from keras.applications import Xception
from keras.utils import multi_gpu_model

import sys, inspect

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

from os.path import expanduser
home = expanduser("~")

sys.path.append(os.path.join( home, "code/DLWorkspace-Utils/keras-multiprocess-image-data-generator") )
import tools.image as T
import multiprocessing
n_process = 16
    
pool = multiprocessing.Pool(processes=n_process)

# clsmembers = inspect.getmembers(sys.modules["keras.applications.inception_v3"], inspect.isclass)
# print( "Classes === %s" % clsmembers)

In [7]:
from IPython.display import SVG
import pydot
from keras.utils.vis_utils import model_to_dot

class Tee(object):
    def __init__(self, name):
        self.file = open(name, "w")
        self.stdout = sys.stdout
        sys.stdout = self
    def __del__(self):
        sys.stdout = self.stdout
        self.file.close()
    def write(self, data):
        self.file.write(data)
        self.stdout.write(data)
    def flush(self):
        self.file.flush()


# dimensions of our images.
#Inception input size
img_width, img_height = 224, 224

top_layers_checkpoint_path = 'cp.top.best.hdf5'
fine_tuned_checkpoint_path = 'cp.fine_tuned.best.hdf5'
new_extended_inception_weights = './logs/final_weights.hdf5'

root_dir = '/data/isic/isic/isic'
data_dir = os.path.join( root_dir, 'isic/2018/task3')

attach_layer = "dense_1"

lamd_final = 0.00
lamd = 0.0
activity_lamd = 0.0






shift = 0.1
zoom = 0.1
# prepare data augmentation configuration
train_datagen = T.ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    rotation_range=90,
    width_shift_range = shift,
    height_shift_range = shift,
    zoom_range=zoom,
    horizontal_flip=True, 
    vertical_flip=True, 
    pool = pool
    )

test_datagen = ImageDataGenerator(rescale=1. / 255, pool = pool )

classes = ["1.MEL","2.NV","3.BCC","4.AKIEC","5.BKL","6.DF","7.VASC"]

nb_train_samples = 8096 
nb_validation_samples = 1536 

from keras.regularizers import * 

import math


from keras.optimizers import *

# opt = SGD(lr = lr_top, decay=0, momentum=0.9, nesterov=True)


train_schedule = [(20, 0.01), (20, 0.001), (20, 0.0001) ]




#Save the TensorBoard logs.
import time
now = time.strftime("%c")

import gc

optimizer_collections = {
    "adadelta" : Adadelta(), 
    "nadam" : Nadam(), 
    "rmsprop": RMSprop(), 
    "adam": Adam(), 
    "adagrad": Adagrad(), 
    "adamax": Adamax(), 
}

if __name__ == "__main__":
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument('--batch_size', default=32, help='Batch size', type=int)
    parser.add_argument('--top_epochs', default=3, help='Number of Epochs(Top training)', type=int)
    parser.add_argument('--nb_epochs', default=60, help='Number of Epochs', type=int)
    parser.add_argument('--optimizer', default='adadelta', help='Optimizer', type=str)
    parser.add_argument('--split', default=None, help='Which data set to use', type=int)
    parser.add_argument('--decay', default=0.0, help='Rate decay', type=float)
    parser.add_argument('--gpus', default=1, help='Number of GPU', type=int)
    parser.add_argument('--drops_epochs', default=0, help='Epochs which rate drop by x10', type=float)
    parser.add_argument('--full_connects', default=1, help='Number of fully connected layer', type = int)

    # parser.add_argument('model', help='Model to evaluate', type=str)
    args = parser.parse_args()

    print( args )
    batch_size = 32
    if args.batch_size:
        batch_size = args.batch_size
    
    num_gpu = args.gpus
    # Number of training epochs
    fit_epochs = args.nb_epochs
    # Number of top epochs
    top_epochs = args.top_epochs
    # data to use. 
    split = args.split
    full_connects = args.full_connects
    
    lr_top = 0.01
    
    if args.optimizer.startswith("sgd"):
        optimizer = args.optimizer
        opt = SGD(lr = lr_top, decay=args.decay, momentum=0.9, nesterov=True)
    else:
        optimizer = args.optimizer
        opt = optimizer_collections[args.optimizer]
        
    
    if split is None:
        parser.print_help()
        exit(-1)
        
    callbacks = []
    
    if args.drops_epochs > 0:
        epochs_drop = args.drops_epochs
        def step_decay(epoch, lr):
            initial_lrate = lr_top
            drop = 0.1
            lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
            # lrate = initial_lrate * math.pow(drop, (1+epoch)/epochs_drop)
            print ("Learning rate at epoch %d === %f " % (epoch, lrate))
            return lrate
        # learning schedule callback
        lrate = LearningRateScheduler(step_decay)
        callbacks.append( lrate )
    
    


NameError: name 'T' is not defined

In [ ]:
if __name__ == "__main__":

    train_data_dir = os.path.join( root_dir, 'isic/2018/task3_split6/split%d/train' % split )
    validation_data_dir = os.path.join( root_dir,'isic/2018/task3_split6/split%d/val' %split )

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        classes = classes,
        shuffle=True,
        class_mode='categorical')

    validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        shuffle=False,
        classes = classes,
        class_mode='categorical')
    
    print(train_generator)
    print(type(train_generator))
    
    # Instantiate the base model (or "template" model).
    # We recommend doing this with under a CPU device scope,
    # so that the model's weights are hosted on CPU memory.
    # Otherwise they may end up hosted on a GPU, which would
    # complicate weight sharing.
    if num_gpu <= 1:
        model_store = "/gpu:0" # use /gpu:0 for single GPU training
    else:
        model_store = "/cpu:0"
    
    with tf.device(model_store):
        # create the base pre-trained model
        base_model = DenseNet201(weights='imagenet', include_top=False)


    model_dir = os.path.join( root_dir, "result/densenet201_%s_batch%s_gpu1_s%d" % ( optimizer, batch_size, split) )
    log_dir = os.path.join( model_dir, "logs")
    os.system( "mkdir -p %s" % log_dir)
    log_file = os.path.join( log_dir, "training_log")
    
    sys.stdout = Tee( log_file )

    import re
    exp = re.compile( "res.*branch.*" )
    with tf.device(model_store):
        for i, layer in enumerate(base_model.layers):
            layer.trainable = False
            if exp.match( layer.name ):
                # print ("Regularize %s" % layer.name)
                layer.kernel_regularizer = l2(lamd)

        # and a logistic layer -- we have 7 classes
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        if full_connects > 0:
            for i in range(full_connects):
                # let's add a fully-connected layer
                x = Dense(1024, activation='relu', kernel_regularizer = l2(lamd_final), activity_regularizer = l1(activity_lamd), trainable = True )(x)
        # and a logistic layer -- let's say we have 200 classes
        predictions = Dense(7, activation='softmax', kernel_regularizer = l2(lamd_final), activity_regularizer = l1(activity_lamd), trainable = True)(x)

        # this is the model we will train
        model = Model(inputs=base_model.input, outputs=predictions)

        # Training new top
        # Replicates the model to multiple GPUs.
        # This assumes that your machine has 8 available GPUs.
    if True:
        if num_gpu <= 1:
            parallel_model = model #multi_gpu_model(model, gpus=num_gpu)
        else:
            parallel_model = multi_gpu_model(model, gpus=num_gpu )
        
        
        parallel_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'] )
        
        # train the model on the new data for a few epochs
        #model.fit_generator(...)
        #class_weights = {"1.MEL":0.082103321,"2.NV":0.013375413,"3.BCC":0.203661327,"4.AKIEC":0.304794521,"5.BKL":0.086914063,"6.DF":1,"7.VASC":0.760683761}
        class_weights = {0:0.082103321,1:0.013375413,2:0.203661327,3:0.304794521,4:0.086914063,5:1,6:0.760683761}

        if top_epochs > 0:
            history = parallel_model.fit_generator(
                train_generator,
                steps_per_epoch=nb_train_samples // batch_size,
                epochs=top_epochs,
                validation_data=validation_generator,
                validation_steps=nb_validation_samples // batch_size,
                class_weight = class_weights,
                callbacks=callbacks)
            del history 
        
        for layer in model.layers:
           layer.trainable = True
        
        # we need to recompile the model for these modifications to take effect
        # we use SGD with a low learning rate
        # opt_fit = SGD(lr = lr_top, decay=0, momentum=0.9, nesterov=True)
        if num_gpu <= 1:
            parallel_model = model #multi_gpu_model(model, gpus=num_gpu)
        else:
            print("Initialize multiple gpu === %d" % num_gpu )
            parallel_model = multi_gpu_model(model, gpus=num_gpu, cpu_merge=False )

        opt_fit = opt
        parallel_model.compile(optimizer=opt_fit, loss='categorical_crossentropy', metrics=['accuracy'])

        # we train our model again (this time fine-tuning the top 2 inception blocks
        # alongside the top Dense layers
        #model.fit_generator(...)

        history = parallel_model.fit_generator(
            train_generator,
            steps_per_epoch=nb_train_samples // batch_size,
            nb_epoch=fit_epochs,
            validation_data=validation_generator,
            validation_steps=nb_validation_samples // batch_size,
            class_weight = class_weights,
            callbacks=callbacks )
        
        model.save(os.path.join(log_dir,'isic_final.h5'))
        
        res = model.predict_generator(validation_generator, steps = nb_validation_samples // batch_size )
        pred = np.argmax(res,axis=1)
        result = T.ClassificationResults(pred, validation_generator.classes, 7)
        print( "Model ==== %s " % model_dir )
        print( "Precision === %s" % np.mean(result.precision) )
        print( "Recall === %s" % np.mean(result.recall) )
        print( "F1 === %s" % np.mean(result.f1) )
        
        # del history
        # del parallel_model 
        # del model
        # del base_model
        # gc.collect()
        # K.clear_session() 
        
        